# COMP809 - Lab 4

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
import statsmodels.api as sm;
import scipy;
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
df = pd.read_csv("framingham.csv");
df = df.dropna(); # eliminate NAs

## Question 1

In [ ]:
# Yes, the data is unbalanced.  We have 85% of 0s and 15% of 1s.

response_count = df.groupby("TenYearCHD")["TenYearCHD"].count();
print(response_count);
print("Percentage of 0s:", 100*response_count[0]/np.sum(response_count));
print("Percentage of 1s:", 100*response_count[1]/np.sum(response_count));

df.describe();

TenYearCHD
0    3099
1     557
Name: TenYearCHD, dtype: int64
Percentage of 0s: 84.76477024070022
Percentage of 1s: 15.23522975929978


### Question 1 (a)

In [ ]:
# Note that if we train the model with this data set, the model could just predict any response
# as a 0, for any predictor values, having an approximated accuracy 0.85.

### Question 1 (b)

In [ ]:
# One of the techniques to deal with unbalanced data is oversampling.
# This method works as follows: those observations with a response value that is minority are
# sampled with replacement M times, where M is the number of observations with a response value
# that is majority.

# Thus, failures and successes are equated

# We could also apply downsampling, in which case observations from the majority group are
# randomly removed to equate this category to the minority group.

## Question 1 (c)

In [ ]:
# Oversampling
df_minority = df[(df['TenYearCHD']==1)];
df_majority = df[(df['TenYearCHD']==0)];
df_minority_upsampled = resample(df_minority,
                                 replace=True,     # sample with replacement
                                 n_samples= response_count[0], # to match majority class
                                 random_state=123);  # reproducible results
df_minority_upsampled.reset_index(drop=True, inplace=True); # reseting row numbers

# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_minority_upsampled, df_majority]);
response_count = df_upsampled.groupby("TenYearCHD")["TenYearCHD"].count();
print(response_count);

# Undersampling -- in case you want to use this option
# downsample majority class
#df_majority_downsampled = resample(df_majority,
#                                 replace=False,    # sample with replacement
#                                 n_samples= response_count[1], # to match minority class
#                                 random_state=123);  # reproducible results

#print(df_minority_upsampled)
#list(df_minority_upsampled.columns)

model  = sm.GLM.from_formula("TenYearCHD ~ C(male) + age + education + C(currentSmoker) + \
                             cigsPerDay + C(BPMeds) + C(prevalentStroke) + C(prevalentHyp) + C(diabetes) + \
                             totChol + sysBP + diaBP + BMI + heartRate+ glucose", family = sm.families.Binomial(),
                             data=df_upsampled);
result = model.fit();
print(result.summary());

# The null deviance shows how well the response is predicted by the model with nothing but an intercept
print(result.null_deviance);
# The residual deviance shows how well the response is predicted by the model when the predictors are included.
print(result.deviance);

# Since there are many continuous predictors, it is highly likely that the responses are ungrouped,
# in which case the overdispersion cannot occur.
# But we will check it anyway.

dev = result.deviance; # Residual Deviance
dof = result.df_resid; # Degree of freedoms of Residuals
pvalue = 1 - scipy.stats.chi2.cdf(dev, dof); # p-value
# H0: Logistic regression model provides an adequate fit for the data
# H1: Logistic regression model does not provide an adequate fit for the data
if pvalue < 0.05:
    print("Saturated model -- p-value: ", pvalue);
else :
    print("Logistic model is ok -- p-value=", pvalue); #

# Rules of thumb

# Calculation of Pearson chi2 / n - (p+1)
print("Pearson2 / Df",result.pearson_chi2 / result.df_resid);
# This value is close to 1

# We also fit a quasi-binomial model
result2 = model.fit(scale="X2");
print(result2.summary());

# The scale parameter is close to 1 in this model

# Conclusion: the logistic regression model provides an adequate fit for the data,
# even though this hypothesis was rejected according to the chi-square test.

# See chaper 13 of Introduction to linear regression analysis
# by Montgomery, Douglas C., author.; Peck, Elizabeth A., author.; Vining, G. Geoffrey, author.
# to know about model selection criteria.


TenYearCHD
0    3099
1    3099
Name: TenYearCHD, dtype: int64
                 Generalized Linear Model Regression Results                  
Dep. Variable:             TenYearCHD   No. Observations:                 6198
Model:                            GLM   Df Residuals:                     6182
Model Family:                Binomial   Df Model:                           15
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -3722.0
Date:                Fri, 22 Mar 2024   Deviance:                       7444.1
Time:                        11:09:51   Pearson chi2:                 6.17e+03
No. Iterations:                     5   Pseudo R-squ. (CS):             0.1691
Covariance Type:            nonrobust                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------

In [ ]:
model  = sm.GLM.from_formula("TenYearCHD ~ C(male) + age + education + C(currentSmoker) + \
                             cigsPerDay + C(BPMeds) + C(prevalentStroke) + C(prevalentHyp) + C(diabetes) + \
                             totChol + sysBP + diaBP + BMI + heartRate+ glucose", family = sm.families.Binomial(),
                             data=df_upsampled);

# Deleting BPMeds since it has the highest p-value
model  = sm.GLM.from_formula("TenYearCHD ~ C(male) + age + education + C(currentSmoker) + \
                             cigsPerDay + C(prevalentStroke) + C(prevalentHyp) + C(diabetes) + \
                             totChol + sysBP + diaBP + BMI + heartRate+ glucose", family = sm.families.Binomial(),
                             data=df_upsampled);

# Deleting heartRate since it has the highest p-value
model  = sm.GLM.from_formula("TenYearCHD ~ C(male) + age + education + C(currentSmoker) + \
                             cigsPerDay + C(prevalentStroke) + C(prevalentHyp) + C(diabetes) + \
                             totChol + sysBP + diaBP + BMI + glucose", family = sm.families.Binomial(),
                             data=df_upsampled);

# Deleting C(currentSmoker) since it has the highest p-value
model  = sm.GLM.from_formula("TenYearCHD ~ C(male) + age + education + \
                             cigsPerDay + C(prevalentStroke) + C(prevalentHyp) + C(diabetes) + \
                             totChol + sysBP + diaBP + BMI + glucose", family = sm.families.Binomial(),
                             data=df_upsampled);

# Deleting diabetes since it has the highest p-value
model  = sm.GLM.from_formula("TenYearCHD ~ C(male) + age + education + \
                             cigsPerDay + C(prevalentStroke) + C(prevalentHyp) +  \
                             totChol + sysBP + diaBP + BMI + glucose", family = sm.families.Binomial(),
                             data=df_upsampled);


result = model.fit();
print(result.summary());

# Just to check the adequacy of the model
result2 = model.fit(scale="X2");
print(result2.summary());
# Note that the scale parameter is close to 1, so the logistic regression model provides an adequate fit for the data

                 Generalized Linear Model Regression Results                  
Dep. Variable:             TenYearCHD   No. Observations:                 6198
Model:                            GLM   Df Residuals:                     6186
Model Family:                Binomial   Df Model:                           11
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -3722.7
Date:                Fri, 22 Mar 2024   Deviance:                       7445.4
Time:                        11:09:55   Pearson chi2:                 6.17e+03
No. Iterations:                     5   Pseudo R-squ. (CS):             0.1689
Covariance Type:            nonrobust                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

## Question 1 (d)

In [ ]:
print('If we increase in one unit the glucose level, the log odds of 10 year risk of coronary heart disease is expected to increase in',\
      round(result.params["glucose"],3), ", holding the other predictors constant.")

print('If we increase in one unit the glucose level, the odds of 10 year risk of coronary heart disease is expected to increase in ',\
      round( np.exp(result.params["glucose"]),3), ", holding the other predictors constant.")

If we increase in one unit the glucose level, the log odds of 10 year risk of coronary heart disease is expected to increase in 0.006 , holding the other predictors constant.
If we increase in one unit the glucose level, the odds of 10 year risk of coronary heart disease is expected to increase in  1.006 , holding the other predictors constant.


## Question 2

In [ ]:
X = df_upsampled.iloc[:, :-1];
y = df_upsampled['TenYearCHD'];

# Here we define training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0,shuffle=True);

aux = pd.concat([X_train, y_train], axis = 1);

model  = sm.GLM.from_formula("TenYearCHD ~ C(male) + age + education + C(currentSmoker) + \
                             cigsPerDay + C(BPMeds) + C(prevalentStroke) + C(prevalentHyp) + C(diabetes) + \
                             totChol + sysBP + diaBP + BMI + heartRate+ glucose", family = sm.families.Binomial(),
                             data=aux);
result = model.fit();
print(result.summary());

### Checking Overdispersion ###

# Since there are many continuous predictors, it is highly likely that the responses are ungrouped,
# in which case the overdispersion cannot occur.
# But we will check it anyway.

dev = result.deviance; # Residual Deviance
dof = result.df_resid; # Degree of freedoms of Residuals
pvalue = 1 - scipy.stats.chi2.cdf(dev, dof); # p-value

# H0: Logistic regression model provides an adequate fit for the data
# H1: Logistic regression model does not provide an adequate fit for the data

if pvalue < 0.05:
    print("Saturated model -- p-value: ", pvalue);
else :
    print("Logistic model is ok -- p-value=", pvalue);

# Rules of thumb

# Calculation of Pearson chi2 / n - (p+1)
print("Pearson2 / Df", result.pearson_chi2 / result.df_resid);
# This value is close to 1.  So the model provides an adequate fit for the data

### Predictions ###
predictions = result.predict(X_test);
predictions_nominal = [ 0 if x < 0.5 else 1 for x in predictions];

from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, predictions_nominal)
print("Confusion matrix: ", cm);
# The diagonal elements of the confusion matrix indicate correct predictions,
#  while the off-diagonals represent incorrect predictions

# The logistic regression correctly predicted the 10 year risk of coronary heart disease 68.87% of the times
print("Accuracy: ", round(np.sum(np.diagonal(cm))/np.sum(cm),3));

# The model correctly predicted 67.7% of the times those with a 10 year risk of coronary heart disease
print("Sensitivity: ", round(cm[1,1]/np.sum(cm[1,:]),3));

# The model correctly predicted 70.7% of the times those without a 10 year risk of coronary heart disease
print("Specificity: ", round(cm[0,0]/np.sum(cm[0,:]),3));

# We can also get those values as follows
print(classification_report(y_test,
                            predictions_nominal,
                            digits = 3))



                 Generalized Linear Model Regression Results                  
Dep. Variable:             TenYearCHD   No. Observations:                 4338
Model:                            GLM   Df Residuals:                     4322
Model Family:                Binomial   Df Model:                           15
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2624.8
Date:                Fri, 22 Mar 2024   Deviance:                       5249.6
Time:                        11:10:00   Pearson chi2:                 4.31e+03
No. Iterations:                     5   Pseudo R-squ. (CS):             0.1615
Covariance Type:            nonrobust                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

## Question 3

In [ ]:
# If multicollinearity is a problem, we can always transform the variables via PCA
# and then fit the logistic regression model.

Xc_train  = X_train.drop(["male","currentSmoker","prevalentStroke","prevalentHyp","diabetes"], axis = 1); # continuous variables

scaler       = StandardScaler();     # Creating object
fitted       = scaler.fit(Xc_train); # Calculating means and SDs
Xc_train_std = fitted.transform(Xc_train); # Standardising data

pca        = PCA(n_components=Xc_train_std.shape[1]); # Specifying number of PCs
pca_fitted = pca.fit(Xc_train_std);                   # Calculating PC transformation
PCAs       = pca_fitted.transform(Xc_train_std);      # Generating PCs
print("Cumulative variability explained by PCs: \n", np.round(np.cumsum(pca.explained_variance_ratio_), 2))

PCs = pd.DataFrame(data = PCAs,
                   columns = ["PC1", "PC2", "PC3", "PC4", "PC5", "PC6", "PC7", "PC8", "PC9", "PC10"])

X_train.reset_index(drop=True, inplace=True) # removing row names
y_train.reset_index(drop=True, inplace=True) # removing row names

DF = pd.concat([y_train, X_train[["male","currentSmoker","prevalentStroke","prevalentHyp","diabetes"]], PCs], axis = 1);

# We fit a linear model with all the principal components and perform back selection

# Model with all the principal components
#model_pca  = sm.GLM.from_formula("TenYearCHD ~ C(male) + C(currentSmoker) + C(prevalentStroke) + C(prevalentHyp) + C(diabetes) +\
#                             PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10", family = sm.families.Binomial(),
#                             data=DF);

# Removing diabetes
#model_pca  = sm.GLM.from_formula("TenYearCHD ~ C(male) + C(currentSmoker) + C(prevalentStroke) + C(prevalentHyp) +\
#                             PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10", family = sm.families.Binomial(),
#                             data=DF);

# Removing currentSmoker
#model_pca  = sm.GLM.from_formula("TenYearCHD ~ C(male) + C(prevalentStroke) + C(prevalentHyp) +\
#                             PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10", family = sm.families.Binomial(),
#                             data=DF);

# Removing PC3
#model_pca  = sm.GLM.from_formula("TenYearCHD ~ C(male) + C(prevalentStroke) + C(prevalentHyp) +\
#                             PC1 + PC2 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10", family = sm.families.Binomial(),
#                             data=DF);

# Removing stroke
model_pca  = sm.GLM.from_formula("TenYearCHD ~ C(male) + C(prevalentHyp) +\
                             PC1 + PC2 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10", family = sm.families.Binomial(),
                             data=DF);

result_pca = model_pca.fit();
print(result_pca.summary());

Xc_test = X_test.drop(["male","currentSmoker","prevalentStroke","prevalentHyp","diabetes"], axis = 1); # continuous variables
Xc_test_std = fitted.transform(Xc_test); # standardised data

X_pca_test = pca_fitted.transform(Xc_test_std);
X_pca_test = pd.DataFrame(data = X_pca_test,
                   columns = ["PC1", "PC2", "PC3", "PC4", "PC5", "PC6", "PC7", "PC8", "PC9", "PC10"])

X_test.reset_index(drop=True, inplace=True) # removing row names
y_test.reset_index(drop=True, inplace=True) # removing row names
X_pca_test = pd.concat([y_test, X_test[["male","currentSmoker","prevalentStroke","prevalentHyp","diabetes"]], X_pca_test], axis = 1);

predictions_pca = result_pca.predict(X_pca_test);
predictions_pca_nominal = [ 0 if x < 0.5 else 1 for x in predictions_pca];

from sklearn.metrics import confusion_matrix, classification_report
cm_pca = confusion_matrix(y_test, predictions_pca_nominal)
print("Confusion matrix: ", cm_pca);
# The diagonal elements of the confusion matrix indicate correct predictions,
#  while the off-diagonals represent incorrect predictions

# The logistic regression correctly predicted the 10 year risk of coronary heart disease 68.87% of the times
print("Acuraccy: ", round(np.sum(np.diagonal(cm_pca))/np.sum(cm_pca),3));

# The model correctly predicted 67.7% of the times those with a 10 year risk of coronary heart disease
print("Sensitivity: ", round(cm[1,1]/np.sum(cm_pca[1,:]),3));

# The model correctly predicted 70.7% of the times those without a 10 year risk of coronary heart disease
print("Specificity: ", round(cm[0,0]/np.sum(cm_pca[0,:]),3));

# We can also get those values as follows
print(classification_report(y_test,
                            predictions_pca_nominal,
                            digits = 3))

# The results are similar to the ones obtain when using the original predictors.
# However, the PC predictors are not correlated.


Cumulative variability explained by PCs: 
 [0.26 0.37 0.48 0.58 0.68 0.77 0.85 0.92 0.98 1.  ]
                 Generalized Linear Model Regression Results                  
Dep. Variable:             TenYearCHD   No. Observations:                 4338
Model:                            GLM   Df Residuals:                     4326
Model Family:                Binomial   Df Model:                           11
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2626.0
Date:                Fri, 22 Mar 2024   Deviance:                       5252.1
Time:                        11:10:03   Pearson chi2:                 4.31e+03
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1610
Covariance Type:            nonrobust                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------